# Process CINIC-10 Dataset


In this notebook, we extract 30,000 images (3,000 for each class) from each of the 10 classes from either CIFAR 10 portion or the ImagineNet portion of the CINIC-10 dataset. This way, we could have trainset and tesetset for same 10 classes, but from different distribution. Resulting a distribution shift.


In [ ]:
import os
import numpy as np
import tarfile
import urllib.request
import random
from PIL import Image
from tqdm import tqdm
import shutil
import os
import matplotlib.pyplot as plt
import pickle

# modify this to set up directory:
DATA_DIR = "/home/data/wangz56"

# add ../../.. to the path (MIAE)
import sys
currdir = os.getcwd()
sys.path.append(os.path.join(currdir, '../../..'))
from miae.utils.dataset_utils import dataset_split
from miae.utils.set_seed import set_seed

seed = 0
set_seed(seed)

path_to_CINIC10 = f'{DATA_DIR}/CINIC10'

In [ ]:
# Function to download and extract CINIC-10 dataset
def download_and_extract_cinic10(url, dest_path):
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    filename = url.split('/')[-1]
    filepath = os.path.join(dest_path, filename)
    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filepath)
        print("Download complete.")
    print(f"Extracting {filename}...")
    with tarfile.open(filepath, "r:gz") as tar:
        tar.extractall(path=dest_path)
    print("Extraction complete.")

    # Function to extract a subset of images from a given dataset path
def extract_subset(dataset_path, num_images_per_class, output_dir):
    # empty output directory
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    classes = os.listdir(dataset_path)
    for c in classes:
        cls_path = os.path.join(dataset_path, c)
        images = os.listdir(cls_path)
        selected_images = random.sample(images, num_images_per_class)
        class_dir = os.path.join(output_dir, c)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        for img in tqdm(selected_images, desc=f"Extracting {num_images_per_class} images from class {c} to {output_dir}"):
            img_path = os.path.join(cls_path, img)
            shutil.copy(img_path, class_dir)

# validate number of images in each class
def validate_num_images_per_class(dataset_path, num_images_per_class):
    classes = os.listdir(dataset_path)
    for c in classes:
        cls_path = os.path.join(dataset_path, c)
        images = os.listdir(cls_path)
        if len(images) < num_images_per_class:
            raise ValueError(f"Class {c} has less than {num_images_per_class} images.")
        
        if len(images) > num_images_per_class:
            print(f"Class {c} has more than {num_images_per_class} images. Randomly selecting {num_images_per_class} images.")
            

In [ ]:
# URLs and paths for CINIC-10 dataset
cinic10_url = 'https://datashare.is.ed.ac.uk/bitstream/handle/10283/3192/CINIC-10.tar.gz?sequence=4&isAllowed=y'
cinic10_path = path_to_CINIC10

# Download and extract CINIC-10 dataset
download_and_extract_cinic10(cinic10_url, cinic10_path)


# Paths for CINIC-10 CIFAR-10 and ImageNet subsets
cinic10_cifar10_train_path = os.path.join(cinic10_path, 'train')
cinic10_imagenet_test_path = os.path.join(cinic10_path, 'test')

# Extract 30,000 images from CINIC-10 CIFAR-10 train set (3,000 per class)
cifar10_train_subset_dir = f'{path_to_CINIC10}/CINIC10_60ksubset/cifar10_train_subset'
extract_subset(cinic10_cifar10_train_path, 3000, cifar10_train_subset_dir)
validate_num_images_per_class(cifar10_train_subset_dir, 3000)

# Extract 30,000 images from CINIC-10 ImageNet test set (3,000 per class)
imagenet_test_subset_dir =  f'{path_to_CINIC10}/CINIC10_60ksubset/imagenet_test_subset'
extract_subset(cinic10_imagenet_test_path, 3000, imagenet_test_subset_dir)
validate_num_images_per_class(imagenet_test_subset_dir, 3000)

print("Dataset preparation complete.")

In [ ]:
import matplotlib.pyplot as plt

# Define the number of rows and columns for the subplots
num_rows = 1
num_cols = 10

# Create the first plot
fig1, axes1 = plt.subplots(num_rows, num_cols, figsize=(20, 4))

# Flatten the axes array
axes1 = axes1.flatten()

train_classes = os.listdir(cifar10_train_subset_dir)

# Iterate over the images and display them as subplots
for i, c in enumerate(train_classes):
    cls_path = os.path.join(cifar10_train_subset_dir, c)
    images = os.listdir(cls_path)
    image_path = os.path.join(cls_path, images[0])
    image = plt.imread(image_path)
    axes1[i].imshow(image)
    axes1[i].set_title(c)
    axes1[i].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Create the second plot
fig2, axes2 = plt.subplots(num_rows, num_cols, figsize=(20, 4))

# Flatten the axes array
axes2 = axes2.flatten()

# Iterate over the images and display them as subplots
for i, c in enumerate(train_classes):
    cls_path = os.path.join(imagenet_test_subset_dir, c)
    images = os.listdir(cls_path)
    image_path = os.path.join(cls_path, images[1])
    image = plt.imread(image_path)
    axes2[i].imshow(image)
    axes2[i].set_title(c)
    axes2[i].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()



In [ ]:
import torch
import torch.utils.data as data
import torchvision.datasets as datasets
from torchvision import transforms as T

mean = [0.47889522, 0.47227842, 0.43047404]
std = [0.24205776, 0.23828046, 0.25874835]
regular_transform = T.Compose([T.ToTensor(),
                                T.Normalize(mean=mean, std=std)
                                ])

augmentation_transform = T.Compose([T.RandomHorizontalFlip(), T.RandomCrop(32, padding=4), T.transforms.ToTensor(),
                                    T.Normalize(mean=mean, std=std)])


class CINIC10(data.Dataset):
    def __init__(self, image_folder, transform=augmentation_transform):
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img, label = self.image_folder[idx]
        return (self.transform(img), label)

dataset_save_dir = f'{DATA_DIR}/miae_experiment_aug_more_target_data/target/cinic10'

cifar10_train_subset_dir = f'{path_to_CINIC10}/CINIC10_60ksubset/cifar10_train_subset'
imagenet_test_subset_dir = f'{path_to_CINIC10}/CINIC10_60ksubset/imagenet_test_subset'

# saving the target_train, target_test, and auxiliary datasets
train_set = CINIC10(datasets.ImageFolder(root=cifar10_train_subset_dir), transform=augmentation_transform)
test_set = CINIC10(datasets.ImageFolder(root=imagenet_test_subset_dir), transform=augmentation_transform)

# build the target_train, target_test, and auxiliary datasets
target_train_len, target_test_len = 15000, len(train_set) - 15000
print(f"target_train_len: {target_train_len}, target_test_len: {target_test_len}")  
target_trainset, target_testset = dataset_split(train_set, [target_train_len, target_test_len], shuffle_seed=seed)
aux_set = test_set

target_membership = np.concatenate([np.ones(len(target_trainset)), np.zeros(len(target_testset))])

# save with pickle
with open(os.path.join(dataset_save_dir, "target_trainset.pkl"), "wb") as f:
    pickle.dump(target_trainset, f)
with open(os.path.join(dataset_save_dir, "target_testset.pkl"), "wb") as f:
    pickle.dump(target_testset, f)
with open(os.path.join(dataset_save_dir, "aux_set.pkl"), "wb") as f:
    pickle.dump(aux_set, f)

dataset_to_attack = data.ConcatDataset([target_trainset, target_testset])
index_to_data = {}
for i in range(len(dataset_to_attack)):
    index_to_data[i] = dataset_to_attack[i]

with open(os.path.join(dataset_save_dir, "index_to_data.pkl"), "wb") as f:
    pickle.dump(index_to_data, f)

np.save(os.path.join(dataset_save_dir, "attack_set_membership.npy"), target_membership)

print(f"Subset saved successfully to {dataset_save_dir}")

In [ ]:
import matplotlib.pyplot as plt
import torchvision

# Load augmented data from CIFAR-10 train set
cifar10_train_augmented = torchvision.datasets.CIFAR10(root=path_to_CINIC10, train=True, transform=augmentation_transform, download=True)

# Load augmented data from CINIC-10 test set
cinic10_test_augmented = torchvision.datasets.ImageFolder(root=cinic10_imagenet_test_path, transform=augmentation_transform)

# Define the number of rows and columns for the subplots
num_rows = 2
num_cols = 10

# Create the subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4))

# Flatten the axes array
axes = axes.flatten()

# Iterate over the CIFAR-10 train augmented data and plot the images
for i in range(num_rows * num_cols):
    image, label = cifar10_train_augmented[i]
    image = image.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
    axes[i].imshow(image)
    axes[i].set_title(cifar10_train_augmented.classes[label])
    axes[i].axis('off')

# Iterate over the CINIC-10 test augmented data and plot the images
for i in range(num_rows * num_cols):
    image, label = cinic10_test_augmented[i]
    image = image.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
    axes[i + num_rows * num_cols].imshow(image)
    axes[i + num_rows * num_cols].set_title(cinic10_test_augmented.classes[label])
    axes[i + num_rows * num_cols].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()